In [1]:
# Import local libraries
import csv_decoder
import save_and_load
import importlib
import numpy as np
import substructure
import matplotlib.pyplot as plt
import time
import pandas as pd
import os

In [2]:
def high_level_feature(higgs_fat_jet, non_higgs_fat_jets, non_higgs_fat_jets_list, length = [0]):
    higgs_pt, higgs_eta, higgs_m = [], [], []
    non_higgs_leading_pt, non_higgs_leading_eta, non_higgs_leading_m = [], [], []
    non_higgs_subleading_pt, non_higgs_subleading_eta, non_higgs_subleading_m = [], [], []
    n_jet = []
    if len(length) == 1:
        element = range(len(higgs_fat_jet))
    else:
         element = length
    for i in element:    
        if higgs_fat_jet[i].pt >= 20:
            higgs_pt.append(higgs_fat_jet[i].pt)
            higgs_eta.append(higgs_fat_jet[i].eta)
            higgs_m.append(higgs_fat_jet[i].mass)
        if non_higgs_fat_jets[i].pt >= 20:
            non_higgs_leading_pt.append(non_higgs_fat_jets[i].pt)
            non_higgs_leading_eta.append(non_higgs_fat_jets[i].eta)
            non_higgs_leading_m.append(non_higgs_fat_jets[i].mass)
        elif non_higgs_fat_jets[i].pt < 20:
            non_higgs_leading_pt.append(-100)
            non_higgs_leading_eta.append(-100)
            non_higgs_leading_m.append(-100)
        if non_higgs_fat_jets_list[i][1].pt >= 20:
            non_higgs_subleading_pt.append(non_higgs_fat_jets_list[i][1].pt)
            non_higgs_subleading_eta.append(non_higgs_fat_jets_list[i][1].eta)
            non_higgs_subleading_m.append(non_higgs_fat_jets_list[i][1].mass)
        elif non_higgs_fat_jets_list[i][1].pt < 20:
            non_higgs_subleading_pt.append(-100)
            non_higgs_subleading_eta.append(-100)
            non_higgs_subleading_m.append(-100)
        n_jet.append(len(non_higgs_fat_jets_list[i]))
        
    return(higgs_pt, higgs_eta, higgs_m, \
           non_higgs_leading_pt, non_higgs_leading_eta, non_higgs_leading_m, \
           non_higgs_subleading_pt, non_higgs_subleading_eta, non_higgs_subleading_m, \
           n_jet)

## Jet Shape Variable

def girth(non_higgs_fat_jets):
    girth = []
    for i in range(len(non_higgs_fat_jets)):
        girth_tmp = 0
        for constit in non_higgs_fat_jets[i]:
            r_tmp = np.sqrt((constit.eta - non_higgs_fat_jets[i].eta)**2 + (constit.phi - non_higgs_fat_jets[i].phi)**2 )
            girth_tmp += (constit.pt*r_tmp)/non_higgs_fat_jets[i].pt
        girth.append(girth_tmp)  
        
    return girth

def central_integrated_jet_shape(non_higgs_fat_jets, r=0.1):
    C_I_J_S = []
    for i in range(len(non_higgs_fat_jets)):
        C_I_J_S_tmp = 0
        for constit in non_higgs_fat_jets[i]:
            r_tmp = np.sqrt((constit.eta - non_higgs_fat_jets[i].eta)**2 + (constit.phi - non_higgs_fat_jets[i].phi)**2 )
            if r_tmp < r:
                C_I_J_S_tmp += (constit.pt)/non_higgs_fat_jets[i].pt
        C_I_J_S.append(C_I_J_S_tmp)  
        
    return C_I_J_S

def side_integrated_jet_shape(non_higgs_fat_jets, r=0.1):
    S_I_J_S = []
    for i in range(len(non_higgs_fat_jets)):
        S_I_J_S_tmp = 0
        for constit in non_higgs_fat_jets[i]:
            r_tmp = np.sqrt((constit.eta - non_higgs_fat_jets[i].eta)**2 + (constit.phi - non_higgs_fat_jets[i].phi)**2 )
            if r_tmp > r and r_tmp < r + 0.1:
                S_I_J_S_tmp += (constit.pt)/non_higgs_fat_jets[i].pt
        S_I_J_S.append(S_I_J_S_tmp)  
        
    return S_I_J_S

In [ ]:
importlib.reload(save_and_load)
# time counter
print(time.strftime("%a %b %d %H:%M:%S %Y", time.localtime()))
ticks_1 = time.time()
############################################################################################################################################################
# save_and_load.load(folder_name, process_name)
ggh_event_list,ggh_mass_list, ggh_higgs_list, ggh_weight_list,ggh_image_list, ggh_recluster_images = save_and_load.load("./numpy_file_2","ggH")
vbf_event_list,vbf_mass_list, vbf_higgs_list, vbf_weight_list, vbf_image_list, vbf_recluster_images = save_and_load.load("./numpy_file_2","VBF")
vh_event_list,vh_mass_list, vh_higgs_list, vh_weight_list, vh_image_list, vh_recluster_images = save_and_load.load("./numpy_file_2","VH")
tth_event_list,tth_mass_list, tth_higgs_list, tth_weight_list, tth_image_list, tth_recluster_images = save_and_load.load("./numpy_file_2","ttH")

############################################################################################################################################################
ticks_2 = time.time()
totaltime =  ticks_2 - ticks_1
print("\033[3;33mTime Cost : {:.4f} min\033[0;m".format(totaltime/60.))

Thu Jun 11 08:33:06 2020


In [ ]:
importlib.reload(csv_decoder)

# time counter
print(time.strftime("%a %b %d %H:%M:%S %Y", time.localtime()))
ticks_1 = time.time()
############################################################################################################################################################

ggh_clustered = csv_decoder.cluster_event(ggh_event_list)
vbf_clustered = csv_decoder.cluster_event(vbf_event_list)
vh_clustered = csv_decoder.cluster_event(vh_event_list)
tth_clustered = csv_decoder.cluster_event(tth_event_list)

ggh_non_higgs_jets, ggh_non_higgs_fat_jets, ggh_higgs_jet, ggh_higgs_fat_jet, ggh_non_higgs_jet_list = csv_decoder.recluster_event(ggh_clustered, ggh_higgs_list)
vbf_non_higgs_jets, vbf_non_higgs_fat_jets, vbf_higgs_jet, vbf_higgs_fat_jet, vbf_non_higgs_jet_list = csv_decoder.recluster_event(vbf_clustered, vbf_higgs_list)
vh_non_higgs_jets, vh_non_higgs_fat_jets, vh_higgs_jet, vh_higgs_fat_jet, vh_non_higgs_jet_list = csv_decoder.recluster_event(vh_clustered, vh_higgs_list)
tth_non_higgs_jets, tth_non_higgs_fat_jets, tth_higgs_jet, tth_higgs_fat_jet, tth_non_higgs_jet_list = csv_decoder.recluster_event(tth_clustered, tth_higgs_list)

print("# of ggH",len(ggh_clustered))
print("# of VBF",len(vbf_clustered))
print("# of VH",len(vh_clustered))
print("# of ttH",len(tth_clustered))
############################################################################################################################################################
ticks_2 = time.time()
totaltime =  ticks_2 - ticks_1
print("\033[3;33mTime consumption : {:.4f} min\033[0;m".format(totaltime/60.))

Thu Jun 11 00:59:36 2020


In [ ]:
# time counter
print(time.strftime("%a %b %d %H:%M:%S %Y", time.localtime()))
ticks_1 = time.time()
############################################################################################################################################################

ggh_higgs_pt, ggh_higgs_eta, ggh_higgs_mass, \
ggh_non_higgs_leading_pt, ggh_non_higgs_leading_eta, ggh_non_higgs_leading_m, \
ggh_non_higgs_subleading_pt, ggh_non_higgs_subleading_eta, ggh_non_higgs_subleading_m, \
n_ggh = high_level_feature(ggh_higgs_fat_jet, ggh_non_higgs_fat_jets, ggh_non_higgs_jet_list)

vbf_higgs_pt, vbf_higgs_eta, vbf_higgs_mass, \
vbf_non_higgs_leading_pt, vbf_non_higgs_leading_eta, vbf_non_higgs_leading_m, \
vbf_non_higgs_subleading_pt, vbf_non_higgs_subleading_eta, vbf_non_higgs_subleading_m, \
n_vbf = high_level_feature(vbf_higgs_fat_jet, vbf_non_higgs_fat_jets, vbf_non_higgs_jet_list)

vh_higgs_pt, vh_higgs_eta, vh_higgs_mass, \
vh_non_higgs_leading_pt, vh_non_higgs_leading_eta, vh_non_higgs_leading_m,  \
vh_non_higgs_subleading_pt, vh_non_higgs_subleading_eta, vh_non_higgs_subleading_m,  \
n_vh = high_level_feature(vh_higgs_fat_jet, vh_non_higgs_fat_jets, vh_non_higgs_jet_list)
    
w_index = np.where(np.array(vh_higgs_list)[:,5] == 24)
wh_higgs_pt, wh_higgs_eta, wh_higgs_mass, \
wh_non_higgs_leading_pt, wh_non_higgs_leading_eta, wh_non_higgs_leading_m,  \
wh_non_higgs_subleading_pt, wh_non_higgs_subleading_eta, wh_non_higgs_subleading_m,  \
n_wh = high_level_feature(vh_higgs_fat_jet, vh_non_higgs_fat_jets, vh_non_higgs_jet_list,w_index[0])
   
z_index = np.where(vh_higgs_list[:,5] == 23)
zh_higgs_pt, zh_higgs_eta, zh_higgs_mass, \
zh_non_higgs_leading_pt, zh_non_higgs_leading_eta, zh_non_higgs_leading_m,  \
zh_non_higgs_subleading_pt, zh_non_higgs_subleading_eta, zh_non_higgs_subleading_m,  \
n_zh = high_level_feature(vh_higgs_fat_jet, vh_non_higgs_fat_jets, vh_non_higgs_jet_list,z_index[0])
    
    
tth_higgs_pt, tth_higgs_eta, tth_higgs_mass, \
tth_non_higgs_leading_pt, tth_non_higgs_leading_eta, tth_non_higgs_leading_m,  \
tth_non_higgs_subleading_pt, tth_non_higgs_subleading_eta, tth_non_higgs_subleading_m,  \
n_tth = high_level_feature(tth_higgs_fat_jet, tth_non_higgs_fat_jets, tth_non_higgs_jet_list)

############################################################################################################################################################
ticks_2 = time.time()
totaltime =  ticks_2 - ticks_1
print("\033[3;33mTime Cost : {:.4f} min\033[0;m".format(totaltime/60.))


In [ ]:
title = ["ggF", "VBF", "VH", "ttH", "WH", "ZH"]
Higgs_PT = [ggh_higgs_pt, vbf_higgs_pt, vh_higgs_pt, tth_higgs_pt]#, wh_higgs_pt, zh_higgs_pt]
Higgs_m = [ggh_higgs_mass, vbf_higgs_mass, vh_higgs_mass, tth_higgs_mass]#, wh_higgs_mass, zh_higgs_mass]
Higgs_eta = [ggh_higgs_eta, vbf_higgs_eta, vh_higgs_eta, tth_higgs_eta]#, wh_higgs_eta, zh_higgs_eta]
non_Higgs_PT = [ggh_non_higgs_leading_pt, vbf_non_higgs_leading_pt, vh_non_higgs_leading_pt, 
                tth_non_higgs_leading_pt]#, wh_non_higgs_leading_pt, zh_non_higgs_leading_pt]
non_Higgs_m = [ggh_non_higgs_leading_m, vbf_non_higgs_leading_m, vh_non_higgs_leading_m, 
               tth_non_higgs_leading_m]#, wh_non_higgs_leading_m, zh_non_higgs_leading_m]
non_Higgs_eta = [ggh_non_higgs_leading_eta, vbf_non_higgs_leading_eta, vh_non_higgs_leading_eta, 
                 tth_non_higgs_leading_eta]#, wh_non_higgs_leading_eta, zh_non_higgs_leading_eta]
non_Higgs_subleading_PT = [ggh_non_higgs_subleading_pt, vbf_non_higgs_subleading_pt, vh_non_higgs_subleading_pt, 
                       tth_non_higgs_subleading_pt]#, wh_non_higgs_subleading_pt, zh_non_higgs_subleading_pt]
non_Higgs_subleading_m = [ggh_non_higgs_subleading_m, vbf_non_higgs_subleading_m, vh_non_higgs_subleading_m, 
                        tth_non_higgs_subleading_m]#, wh_non_higgs_subleading_m, zh_non_higgs_subleading_m]
non_Higgs_subleading_eta = [ggh_non_higgs_subleading_eta, vbf_non_higgs_subleading_eta, vh_non_higgs_subleading_eta, 
                        tth_non_higgs_subleading_eta]#, wh_non_higgs_subleading_eta, zh_non_higgs_subleading_eta]
n_jet = [n_ggh,n_vbf,n_vh, n_tth]#, n_wh, n_zh]



fig = plt.figure(figsize=(10,8))
for i, process in enumerate(Higgs_PT):
    length = np.linspace(350,1200,160)
    hist, bins = np.histogram(process, bins=length)
    plt.step(bins[:-1], hist.astype(np.float32) / hist.sum(), where='mid',linewidth=2, alpha=0.7, label=title[i])
#     plt.hist(process, 100, facecolor='blue', label=title[i],histtype = 'step')
    plt.legend(ncol=1,fontsize=20, edgecolor = "w")
    plt.title("Higgs Jet PT", fontsize = 25)
    plt.xlabel("$PT$ [GeV]", fontsize=30,horizontalalignment='right',x=1) 
    
plt.ylabel("1/N /5GeV",fontsize=30,horizontalalignment='right',y=1)   
plt.show()
plt.savefig("./Higgs_Pt/higgs_j_pt")

fig = plt.figure(figsize=(10,8))

for i, process in enumerate(Higgs_m):
    length = np.linspace(0,200,201)
    hist, bins = np.histogram(process, bins=length)
    plt.step(bins[:-1], hist.astype(np.float32) / hist.sum(), where='mid',linewidth=2, alpha=0.7, label=title[i])
#     plt.hist(process, 100, facecolor='blue', label=title[i],histtype = 'step')
    plt.legend(ncol=1,fontsize=20, edgecolor = "w")
    plt.title("Higgs Jet Mass", fontsize = 25)
    plt.xlabel("$M_J$ [GeV]", fontsize=30,horizontalalignment='right',x=1) 
    
plt.xlim((50,200))
plt.ylabel("1/N /1GeV",fontsize=30,horizontalalignment='right',y=1)   
# plt.show()
plt.savefig("./Higgs_Pt/higgs_j_m")

fig = plt.figure(figsize=(10,8))
for i, process in enumerate(Higgs_eta):
    length = np.linspace(-5,5,50)
    hist, bins = np.histogram(process, bins=length)
    plt.step(bins[:-1], hist.astype(np.float32) / hist.sum(), where='mid',linewidth=2, alpha=0.7, label=title[i])
#     plt.hist(process, 100, facecolor='blue', label=title[i],histtype = 'step')
    plt.legend(ncol=1,fontsize=20, edgecolor = "w")
    plt.title("Higgs Jet $\eta$", fontsize = 25)
    plt.xlabel("$\eta$ ", fontsize=30,horizontalalignment='right',x=1) 
    
plt.ylabel("1/N",fontsize=30,horizontalalignment='right',y=1)   
# plt.show()
plt.savefig("./Higgs_Pt/higgs_j_eta")

fig = plt.figure(figsize=(10,8))
for i, process in enumerate(non_Higgs_PT):
    length = np.linspace(0,1500,300)
    hist, bins = np.histogram(process, bins=length)
    plt.step(bins[:-1], hist.astype(np.float32) / hist.sum(), where='mid',linewidth=2, alpha=0.7, label=title[i])
#     plt.hist(process, 100, facecolor='blue', label=title[i],histtype = 'step')
    plt.legend(ncol=1,fontsize=20, edgecolor = "w")
#     plt.title("non-Higgs Leading Jet PT", fontsize = 25)
    plt.xlabel("$PT_j$ [GeV]", fontsize=30,horizontalalignment='right',x=1) 
    
plt.ylabel("1/N /5GeV",fontsize=30,horizontalalignment='right',y=1)   
# plt.show()
plt.savefig("./Higgs_Pt/non_higgs_j_pt")

fig = plt.figure(figsize=(10,8))

for i, process in enumerate(non_Higgs_m):
    length = np.linspace(0,300,150)
    hist, bins = np.histogram(process, bins=length)
    plt.step(bins[:-1], hist.astype(np.float32) / hist.sum(), where='mid',linewidth=2, alpha=0.7, label=title[i])
#     plt.hist(process, 100, facecolor='blue', label=title[i],histtype = 'step')
    plt.legend(ncol=1,fontsize=20, edgecolor = "w")
#     plt.title("non-Higgs Leading Jet Mass", fontsize = 25)
    plt.xlabel("$M_j$ [GeV]", fontsize=30,horizontalalignment='right',x=1) 

plt.xlim((0,300))
plt.ylabel("1/N /2GeV",fontsize=30,horizontalalignment='right',y=1)   
# plt.show()
plt.savefig("./Higgs_Pt/non_higgs_j_m")

fig = plt.figure(figsize=(10,8))
for i, process in enumerate(non_Higgs_eta):
    length = np.linspace(-5,5,50)
    hist, bins = np.histogram(process, bins=length)
    plt.step(bins[:-1], hist.astype(np.float32) / hist.sum(), where='mid',linewidth=2, alpha=0.7, label=title[i])
#     plt.hist(process, 100, facecolor='blue', label=title[i],histtype = 'step')
    plt.legend(ncol=1,fontsize=20, edgecolor = "w")
#     plt.title("non-Higgs Leading Jet $\eta$", fontsize = 25)
    plt.xlabel("$\eta_j$ ", fontsize=30,horizontalalignment='right',x=1) 
    
plt.ylabel("1/N",fontsize=30,horizontalalignment='right',y=1)   
# plt.show()
plt.savefig("./Higgs_Pt/non_higgs_j_eta")


fig = plt.figure(figsize=(10,8))
for i, process in enumerate(n_jet):
    plt.hist(process, 100, range=(0,100), facecolor='blue', label=title[i],histtype = 'step')
    plt.legend(ncol=1,fontsize=20, edgecolor = "w")
    plt.title("# of non-Higgs Jet", fontsize = 25)
    plt.xlabel("", fontsize=30,horizontalalignment='right',x=1) 
    
# plt.xlim((0,30))
plt.ylabel("#",fontsize=30,horizontalalignment='right',y=1)   
# plt.show()
plt.savefig("./Higgs_Pt/N_non_higgs_j")


fig = plt.figure(figsize=(10,8))
for i, process in enumerate(non_Higgs_subleading_PT):
    length = np.linspace(0,1500,300)
    hist, bins = np.histogram(process, bins=length)
    plt.step(bins[:-1], hist.astype(np.float32) / hist.sum(), where='mid',linewidth=2, alpha=0.7, label=title[i])
#     plt.hist(process, 100, facecolor='blue', label=title[i],histtype = 'step')
    plt.legend(ncol=1,fontsize=20, edgecolor = "w")
    plt.title("non-Higgs sub-Leading Jet PT", fontsize = 25)
    plt.xlabel("$PT$ [GeV]", fontsize=30,horizontalalignment='right',x=1) 
    
plt.ylabel("1/N /5GeV",fontsize=30,horizontalalignment='right',y=1)   
# plt.show()
# plt.savefig("./non_higgs_subleading_j_pt")

fig = plt.figure(figsize=(10,8))
for i, process in enumerate(non_Higgs_subleading_m):
    length = np.linspace(0,300,150)
    hist, bins = np.histogram(process, bins=length)
    plt.step(bins[:-1], hist.astype(np.float32) / hist.sum(), where='mid',linewidth=2, alpha=0.7, label=title[i])
#     plt.hist(process, 100, facecolor='blue', label=title[i],histtype = 'step')
    plt.legend(ncol=1,fontsize=20, edgecolor = "w")
    plt.title("non-Higgs sub-Leading Jet Mass", fontsize = 25)
    plt.xlabel("$M_J$ [GeV]", fontsize=30,horizontalalignment='right',x=1) 

plt.xlim((0,200))
plt.ylabel("1/N /2GeV",fontsize=30,horizontalalignment='right',y=1)   
# plt.show()
plt.savefig("./Higgs_Pt/non_higgs_j_m")


    
fig = plt.figure(figsize=(10,8))
for i, process in enumerate(non_Higgs_subleading_eta):
    length = np.linspace(-5,5,50)
    hist, bins = np.histogram(process, bins=length)
    plt.step(bins[:-1], hist.astype(np.float32) / hist.sum(), where='mid',linewidth=2, alpha=0.7, label=title[i])
#     plt.hist(process, 100, facecolor='blue', label=title[i],histtype = 'step')
    plt.legend(ncol=1,fontsize=20, edgecolor = "w")
    plt.title("non-Higgs sub-Leading Jet $\eta$", fontsize = 25)
    plt.xlabel("$\eta$ ", fontsize=30,horizontalalignment='right',x=1) 
    
plt.ylabel("1/N",fontsize=30,horizontalalignment='right',y=1)   
# plt.show()
plt.savefig("./Higgs_Pt/non_higgs_subleading_j_eta")



### We are not in 2-jet Category, so $|\Delta \eta_{j_1,j_2}|$ is hard to make.

In [ ]:
non_Higgs_eta = [np.array(ggh_non_higgs_leading_eta), np.array(vbf_non_higgs_leading_eta), np.array(vh_non_higgs_leading_eta), np.array(tth_non_higgs_leading_eta)]
non_Higgs_subleading_eta = [np.array(ggh_non_higgs_subleading_eta), np.array(vbf_non_higgs_subleading_eta), np.array(vh_non_higgs_subleading_eta), np.array(tth_non_higgs_subleading_eta)]


fig = plt.figure(figsize=(10,8))
for i, process in enumerate(abs(np.array(non_Higgs_subleading_eta)-np.array(non_Higgs_eta))):
    for j in range(len(process)):
        if process[j] > 50:
            process[j] = -1
    length = np.linspace(-10,10,80)
    hist, bins = np.histogram(process, bins=length)
    plt.step(bins[:-1], hist.astype(np.float32) / hist.sum(), where='mid',linewidth=2, alpha=0.7, label=title[i])
#     plt.hist(process, 100, facecolor='blue', label=title[i],histtype = 'step')
    plt.legend(ncol=1,fontsize=20, edgecolor = "w")
#     plt.title("$\Delta\eta_{jj}$", fontsize = 25)
    plt.xlabel("$|\Delta\eta_{jj}|$", fontsize=30,horizontalalignment='right',x=1) 
    
plt.xlim((0,10)) 
plt.ylim((0,0.12)) 
plt.ylabel("1/N",fontsize=30,horizontalalignment='right',y=1)   
plt.savefig("./Higgs_Pt/delta_eta_eta")
plt.show()


# fig = plt.figure(figsize=(10,8))
# for i, process in enumerate(non_Higgs_subleading_eta):
#     length = np.linspace(-5,5,50)
#     hist, bins = np.histogram(process, bins=length)
#     plt.step(bins[:-1], hist.astype(np.float32) / hist.sum(), where='mid',linewidth=2, alpha=0.7, label=title[i])
# #     plt.hist(process, 100, facecolor='blue', label=title[i],histtype = 'step')
#     plt.legend(ncol=1,fontsize=20, edgecolor = "w")
#     plt.title("non-Higgs sub-Leading Jet $\eta$", fontsize = 25)
#     plt.xlabel("$\eta$ ", fontsize=30,horizontalalignment='right',x=1) 
    
# plt.ylabel("1/N",fontsize=30,horizontalalignment='right',y=1)   
# # plt.show()
# # plt.savefig("./non_higgs_subleading_j_eta")


In [ ]:
GIRTH = [girth(ggh_non_higgs_fat_jets),girth(vbf_non_higgs_fat_jets),girth(vh_non_higgs_fat_jets), girth(tth_non_higgs_fat_jets)]
C_I_J_S = [central_integrated_jet_shape(ggh_non_higgs_fat_jets),central_integrated_jet_shape(vbf_non_higgs_fat_jets),central_integrated_jet_shape(vh_non_higgs_fat_jets),central_integrated_jet_shape(tth_non_higgs_fat_jets)]        
S_I_J_S = [side_integrated_jet_shape(ggh_non_higgs_fat_jets),side_integrated_jet_shape(vbf_non_higgs_fat_jets),side_integrated_jet_shape(vh_non_higgs_fat_jets),side_integrated_jet_shape(tth_non_higgs_fat_jets)]        


In [ ]:
title = ["ggH", "VBF", "VH", "ttH"]
substructure_name = ["girth", "$\Psi_c$", "$\Psi_s$"]
substructure = [GIRTH, C_I_J_S, S_I_J_S]

for name, element in enumerate(substructure):
    fig, ax = plt.subplots(1,1, figsize=(10,8))
    for i, process in enumerate(element):
        length = np.linspace(0,1,100)
        hist, bins = np.histogram(process, bins=length)
        ax.step(bins[:-1], hist.astype(np.float32) / hist.sum(), where='mid',linewidth=2, alpha=0.7, label=title[i])
    if substructure_name[name] == "girth":
        plt.xlim((0,0.4))
        
#     plt.title(substructure_name[name], fontsize = 25)
    plt.legend(ncol=1,fontsize=20, edgecolor = "w")
    plt.xlabel(substructure_name[name], fontsize=25,horizontalalignment='right',x=1) 
    plt.ylabel("1/N",fontsize=25,horizontalalignment='right',y=1)   
    plt.savefig("./Higgs_Pt/"+substructure_name[name])
    plt.show()



In [ ]:
def Save_Features(higgs_pt, higgs_eta, higgs_mass, \
            non_higgs_leading_pt, non_higgs_leading_eta, non_higgs_leading_m, \
            non_higgs_subleading_pt, non_higgs_subleading_eta, non_higgs_subleading_m, \
            girth,CIJS,SIJS, weight_list,\
            index):
    
    if index == "isGGH":
        isGGH, isVBF, isVH, isttH = 1, 0, 0, 0
    elif index == "isVBF":
        isGGH, isVBF, isVH, isttH = 0, 1, 0, 0
    elif index == "isVH":
        isGGH, isVBF, isVH, isttH = 0, 0, 1, 0
    elif index == "isttH":
        isGGH, isVBF, isVH, isttH = 0, 0, 0, 1
        
    features = ["higgs_pt","higgs_eta","higgs_m",
           "non_higgs_leading_pt","non_higgs_leading_eta","non_higgs_leading_m",
          # "non_higgs_subleading_pt","non_higgs_subleading_eta","non_higgs_subleading_m",
           "girth","CIJS","SIJS","DeltaEta",
           "weight","isGGH","isVBF","isVH","isttH","eventindex"]
    
    if os.path.exists("./High_Level_Features") == 0:
        os.mkdir("./High_Level_Features")
    path = "./High_Level_Features/"

    counter = 0
    for j, filename in enumerate(os.listdir(path)):
        if filename == "higgs_highlevelfeatures.csv":
            counter = 1 
            
    if counter == 0:
        dataframe = pd.DataFrame()
    if counter == 1:
        dataframe = pd.DataFrame()
        save_to_csvdata = pd.read_csv(path + "higgs_highlevelfeatures.csv")

    for i in range(len(higgs_pt)):
        var = []
        var.append(higgs_pt[i])
        var.append(higgs_eta[i])
        var.append(higgs_mass[i])
        var.append(non_higgs_leading_pt[i])
        var.append(non_higgs_leading_eta[i])
        var.append(non_higgs_leading_m[i])
#         var.append(non_higgs_subleading_pt[i])
#         var.append(non_higgs_subleading_eta[i])
#         var.append(non_higgs_subleading_m[i])
        var.append(girth[i])
        var.append(CIJS[i])
        var.append(SIJS[i])
        
        deltaeta = abs(np.array(non_higgs_subleading_eta[i])-np.array(non_higgs_leading_eta[i]))
        if deltaeta > 50:
            var.append(-1)
        else:
            var.append(deltaeta)
        
        var.append(weight_list[i])
        var.append(isGGH)
        var.append(isVBF)
        var.append(isVH)
        var.append(isttH)
        var.append(i)
        dataframe_tmp = pd.DataFrame([var],columns=features)
        dataframe = dataframe.append(dataframe_tmp,ignore_index=True)

    if counter == 0:
        dataframe.to_csv( path + "higgs_highlevelfeatures.csv",index = 0)
    elif counter == 1:
        DATA = pd.concat([save_to_csvdata,dataframe],ignore_index=True,axis=0,join='inner')
        DATA.to_csv(path + "higgs_highlevelfeatures.csv",index = 0)

In [13]:
Save_Features(ggh_higgs_pt, ggh_higgs_eta, ggh_higgs_mass, \
       ggh_non_higgs_leading_pt, ggh_non_higgs_leading_eta, ggh_non_higgs_leading_m, \
       ggh_non_higgs_subleading_pt, ggh_non_higgs_subleading_eta, ggh_non_higgs_subleading_m, \
       GIRTH[0],C_I_J_S[0],S_I_J_S[0],ggh_weight_list, "isGGH")
Save_Features(vbf_higgs_pt, vbf_higgs_eta, vbf_higgs_mass, \
       vbf_non_higgs_leading_pt, vbf_non_higgs_leading_eta, vbf_non_higgs_leading_m, \
       vbf_non_higgs_subleading_pt, vbf_non_higgs_subleading_eta, vbf_non_higgs_subleading_m, \
       GIRTH[1],C_I_J_S[1],S_I_J_S[1],vbf_weight_list, "isVBF")
Save_Features(vh_higgs_pt, vh_higgs_eta, vh_higgs_mass, \
       vh_non_higgs_leading_pt, vh_non_higgs_leading_eta, vh_non_higgs_leading_m, \
       vh_non_higgs_subleading_pt, vh_non_higgs_subleading_eta, vh_non_higgs_subleading_m, \
       GIRTH[2],C_I_J_S[2],S_I_J_S[2],vh_weight_list, "isVH")
Save_Features(tth_higgs_pt, tth_higgs_eta, tth_higgs_mass, \
       tth_non_higgs_leading_pt, tth_non_higgs_leading_eta, tth_non_higgs_leading_m, \
       tth_non_higgs_subleading_pt, tth_non_higgs_subleading_eta, tth_non_higgs_subleading_m, \
       GIRTH[3],C_I_J_S[3],S_I_J_S[3],tth_weight_list, "isttH")
        